## <a name="0">Inicio</a>

- [1. Importaciones necesarias](#1)

- [2. Diccionario de IDs de los municipios del analisis](#2)

- [3. Variables Economicas (REMESAS, COMERCIO EXT e INT, )](#3)
    *   [3.1. EDA Remesas](#3.1)
    *   [3.2. EDA Comercio Exterior e Interior](#3.2)

- [4. Graficos Variables Economicas Por Municipio](#4)




## <a name="1">1. Importaciones necesarias</a>

([Ir al inicio](#0))


In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

## <a name="2">2. Diccionario de IDs de los municipios del analisis</a>

([Ir al inicio](#0))

In [5]:
# Diccionario de municipios con sus IDs
municipios_ids = {
    'Hermosillo': 26030,
    'Cajeme': 26018,
    'Nogales': 26043,
    'San Luis Río Colorado': 26055,
    'Navojoa': 26042,
    'Magdalena de Kino': 26036,
    'Agua Prieta': 26002,
    'Caborca': 26017,
    'Puerto Peñasco': 26048,
    'Huatabampo': 26033
}

## <a name="3">3. Variables Economicas (REMESAS, COMERCIO EXT e INT, )</a>

([Ir al inicio](#0))

In [9]:
# ================ Variables Economicas ===================

# Lista para almacenar todos los DataFrames
remesas = []

# Loop por cada municipio
for nombre, id_mun in municipios_ids.items():
    url = f"http://www.economia.gob.mx/datamexico/api/data.jsonrecords?Municipality={id_mun}&cube=banxico_mun_income_remittances&drilldowns=Municipality,Quarter&measures=Remittance+Amount&locale=es"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("data", [])
        df = pd.DataFrame(data)[["Municipality", "Quarter", "Remittance Amount"]]
        remesas.append(df)
    else:
        print(f"Error al obtener datos de {nombre}")

# Concatenar todos los DataFrames
df_remesas = pd.concat(remesas, ignore_index=True)

# Lista para almacenar todos los DataFrames
comercio_ext = []

# Loop por cada municipio
for nombre, id_mun in municipios_ids.items():
    url = f"http://www.economia.gob.mx/datamexico/api/data?Municipality={id_mun}&Product+Level=2&cube=economy_foreign_trade_mun&drilldowns=Month,Flow&measures=Trade+Value&parents=false&locale=es"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("data", [])
        if data:
            df = pd.DataFrame(data)
            columnas = ["Month", "Flow", "Trade Value"]
            df = df[[col for col in columnas if col in df.columns]]
            df.insert(0, "Municipality", nombre)  # Insertar como primera columna
            comercio_ext.append(df)
        else:
            print(f"Sin datos para {nombre}")
    else:
        print(f"Error al obtener datos de {nombre} (status {response.status_code})")

# Concatenar todos los DataFrames
df_comercio_ext = pd.concat(comercio_ext, ignore_index=True)

## <a name="3.1">3.1 EDA Remesas</a>

([Ir al inicio](#0))

In [21]:
# Imprimir las primeras cinco filas
df_remesas.head(20)

,Municipality,Quarter,Remittance Amount
0,Hermosillo,2013-Q1,17967135
1,Hermosillo,2013-Q2,19585900
2,Hermosillo,2013-Q3,18196749
3,Hermosillo,2013-Q4,17406996
4,Hermosillo,2014-Q1,17268599
5,Hermosillo,2014-Q2,19477728
6,Hermosillo,2014-Q3,19067498
7,Hermosillo,2014-Q4,16762738
8,Hermosillo,2015-Q1,19700121
9,Hermosillo,2015-Q2,28937968


In [12]:
# Veamos los tipos de datos y valores no nulos para cada columna
df_remesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Municipality       500 non-null    object
 1   Quarter            500 non-null    object
 2   Remittance Amount  500 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.8+ KB


In [13]:
# Esto imprime estadísticas básicas para todas las columnas numéricas
df_remesas.describe()

,Remittance Amount
count,5.000000e+02
mean,1.284018e+07
std,1.315120e+07
min,7.784520e+05
25%,4.370993e+06
50%,8.371534e+06
75%,1.782838e+07
max,7.377033e+07


## <a name="3.2">3.2 EDA Comercio exterior e interior</a>

([Ir al inicio](#0))

In [15]:
# Imprimir las primeras cinco filas
df_comercio_ext.head()

,Municipality,Month,Flow,Trade Value
0,Hermosillo,2006-01,Compras internacionales,86629925
1,Hermosillo,2006-01,Ventas internacionales,64610295
2,Hermosillo,2006-02,Compras internacionales,93366546
3,Hermosillo,2006-02,Ventas internacionales,66776754
4,Hermosillo,2006-03,Compras internacionales,110531356


In [16]:
# Veamos los tipos de datos y valores no nulos para cada columna
df_comercio_ext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3974 entries, 0 to 3973
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Municipality  3974 non-null   object
 1   Month         3974 non-null   object
 2   Flow          3974 non-null   object
 3   Trade Value   3974 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 124.3+ KB


In [17]:
# Esto imprime estadísticas básicas para todas las columnas numéricas
df_comercio_ext.describe()

,Trade Value
count,3.974000e+03
mean,7.448062e+07
std,1.037742e+08
min,8.200000e+01
25%,4.667893e+06
50%,3.041973e+07
75%,8.622142e+07
max,5.802366e+08


## <a name="4">4. Graficos Variables Economicas Por Municipio</a>

([Ir al inicio](#0))

In [19]:
# ================ Grafica x 10 Remesas netas ===================

# Crear un gráfico por municipio
for municipio in df_total["Municipality"].unique():
    df_mun = df_total[df_total["Municipality"] == municipio].sort_values("Quarter")
    plt.figure(figsize=(10, 5))
    plt.plot(df_mun["Quarter"], df_mun["Remittance Amount"], marker='o')
    plt.title(f"Remesas por trimestre - {municipio}")
    plt.xlabel("Trimestre")
    plt.ylabel("Monto de remesas (MXN)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ================ Grafica x 10 Comercio internacional neto ===================

# Asegurar que las fechas estén ordenadas correctamente
df_comercio_ext["Month"] = pd.to_datetime(df_comercio_ext["Month"], format="%Y-%m")

# Crear un gráfico por municipio
for municipio in df_comercio_ext["Municipality"].unique():
    df_mun = df_comercio_ext[df_comercio_ext["Municipality"] == municipio]

    plt.figure(figsize=(12, 6))

    # Graficar cada tipo de flujo por separado
    for flujo in df_mun["Flow"].unique():
        df_flujo = df_mun[df_mun["Flow"] == flujo].sort_values("Month")
        plt.plot(df_flujo["Month"], df_flujo["Trade Value"], label=flujo, marker='o')

    plt.title(f"Comercio exterior mensual - {municipio}")
    plt.xlabel("Mes")
    plt.ylabel("Valor comercial (MXN)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.xticks(rotation=45)
    plt.show()

NameError: name 'df_total' is not defined